# Preliminary phase

### Import of the libraries

In [74]:
# Import the basic spark library
from pyspark.sql import SparkSession
from pyspark.sql.functions import when, lit, collect_list, col, regexp_replace, size, expr, count, countDistinct, array_contains, explode, avg

### Session creation

In [75]:
# Create an entry point to the PySpark Application
spark = SparkSession.builder \
    .master("local") \
    .appName("MyFirstSparkApplication") \
    .getOrCreate()
# master contains the URL of your remote spark instance or 'local'

### Vettori (?)

In [76]:
vectASIA = ["Department of Statistics, Rajshahi University, Rajshahi, Bangladesh",
            "Department of Computer Science & Engineering, BUET, Dhaka, Bangladesh",
            'Department of System Design and Informatics, Kyushu Institute of Technology, Fukuka, Japan',
            "Corporate Research and Development Center, Toshiba Corporation, Kawasaki, Japan",
            "Institute of Industrial Science, University of Tokyo, Meguro-Ku, Japan", "continent",
            "Department of Human and Artificial Intelligence System, Graduate School of Engineering, University of Fukui, Japan",
            "Department of Computer Science, Faculty of Engineering, Yamanashi University, Kofu, Japan",
            "Nippon Hoso Kyokai", "Shinshu University", 'Nagano Prefectural College']

vectEU = ["Università della Calabria", "Università degli Studi di Modena e Reggio Emilia", "Politecnico di Milano",
          'Università degli Studi di Firenze'
          "University of Milano-Bicocca",
          "Cognitive Interaction Technology, Center of Excellence and Applied Informatics, Faculty of Technology, Bielefeld University, Bielefeld, Germany#TAB#",
          "Department of Information Systems and Business Administration, Johannes Gutenberg-Universität Mainz, Jakob-Welder-Weg 9, Mainz 55128, Germany",
          "Dept. of Computer Science, Uni. Politècnica de Catalunya, Spain#TAB#", "university of malaga",
          "Computer Laboratory, University Of Cambridge", "New England Aquarium",
          "Unconventional Computing Group, University of the West of England, Bristol, UK BS16 1QY#TAB#",
          "ESIEE-Paris", "UNIVERSITÉ LAVAL", "University of Southern Denmark", "University of Lisbon",
          "#N#University of Glasgow#N#"]

vectAMERICA = ['The University of Chicago Press, Chicago, USA']

# Data upload

### Load from files

In [77]:
# Non so se è giusto, gli id vengono letti come nullable = true (dovrebbe essere false essendo id)

author_df = spark.read.json("authors.json")
fos_df = spark.read.json("fos.json")
publication_df = spark.read.json("publications.json")
venue_df = spark.read.json("venues.json")
rel_df = spark.read.json("rel_dw.json")

### Tables showing

In [78]:
# # Authors
# print("Tabella autori ")
# # author_df.show(truncate=True)
# author_df.printSchema()

In [79]:
# FOS
# print("Tabella fos")
# fos_df.show(truncate=True)

In [80]:
# Publications
# print("Tabella publication")
# publication_df.show(truncate=True)

In [81]:
# # Venues
# print("Tabella venue")
# venue_df.show(truncate=True)

In [82]:
# Relation Pub-FOS
# print("Tabella rel")
# rel_df.show(truncate=False)

# Queries

### Creation/Update

In [83]:
# Q1 - Creation of a row
# columns = ["date", "id", "number", "raw", "type", "volume"]
# newRow = spark.createDataFrame([("2022-12-05", 9999999, 99, "New venue raw", "C", 3)], columns)
# venue_df = venue_df.union(newRow)
# venue_df.show(truncate=False)

In [84]:
# Q2 - Creation of the new column that counts the number of references
# publication_df.withColumn("references number", size(col("references"))).show(truncate=False)

In [85]:
# Q3 - Creation of a new column that represents the region of the university
# author_df = author_df.withColumn('Continent',
#                                  when(author_df.affiliation.isin(vectASIA), lit("Asia"))
#                                  .when(author_df.affiliation.isin(vectAMERICA), lit("America"))
#                                  .when(author_df.affiliation.isin(vectEU), lit("Europa"))
#                                  .otherwise(lit("Rest of the world"))
#                                  ).show(truncate=True)

In [86]:
# Q4 - Update of fos names
# fos_df = fos_df.withColumn('name',
#                            when(fos_df.name.contains("mathematics"),
#                                 regexp_replace(fos_df.name, 'Discrete mathematics', 'Discrete Math'))
#                            .when(fos_df.name.contains("Mathematics"),
#                                  regexp_replace(fos_df.name, 'Mathematics', 'Math'))
#                            .when(fos_df.name.contains("Artificial intelligence"),
#                                  regexp_replace(fos_df.name, 'Artificial intelligence', 'AI'))
#                            .otherwise(fos_df.name)).show()

In [87]:
 # # Q5 - Delete every publication whose type is Conference - da cambiare
# publication_df = publication_df.where(publication_df.type != "Conference").show()

### Other queries

Descrizione query

In [88]:
# WHERE, JOIN
fos_df.filter(col("name") == "Artificial intelligence") \
.join(rel_df, fos_df.id == rel_df.fos_id, "inner") \
.join(publication_df, rel_df.pub_id == publication_df.id, "inner") \
.select("title").show(truncate=False)

+--------------------------------------------------------------------------------------------------+
|title                                                                                             |
+--------------------------------------------------------------------------------------------------+
|A methodology for the physically accurate visualisation of roman polychrome statuary              |
|Comparison of GARCH, Neural Network and Support Vector Machine in Financial Time Series Prediction|
|COMPARING GNG3D AND QUADRIC ERROR METRICS METHODS TO SIMPLIFY 3D MESHES                           |
|Vectorial fast correlation attacks.                                                               |
|Improved Secret Image Sharing Method By Encoding Shared Values With Authentication Bits           |
|Identifying Psychological Theme Words from Emotion Annotated Interviews                           |
|A COMPUTATIONAL SALIENCY MODEL INTEGRATING SACCADE PROGRAMMING                            

Descrizione query

In [89]:
# WHERE, LIMIT, LIKE
author_df.withColumnRenamed("id", "authorId").filter(col("affiliation").like("%Politecnico%")).limit(5).join(
    publication_df, expr(
        "array_contains(authors, authorId)")).select(col("title").alias("publicationTitle"),
                                                     col("name").alias("authorName"), "affiliation").show(truncate=False)
author_df.withColumnRenamed("id", "authorId").filter(
    col("affiliation").like("%Politecnico%")).limit(5)
#publication_df.select(count(.name)).show()

+--------------------------------------------------------------+-----------------+---------------------+
|publicationTitle                                              |authorName       |affiliation          |
+--------------------------------------------------------------+-----------------+---------------------+
|Vectorial fast correlation attacks.                           |Jovan Dj. Golic  |Politecnico di Milano|
|Vectorial fast correlation attacks.                           |Guglielmo Morgari|Politecnico di Bari  |
|A COMPUTATIONAL SALIENCY MODEL INTEGRATING SACCADE PROGRAMMING|Tien Ho-Phuoc    |Politecnico di Milano|
|Cleaneval: a Competition for Cleaning Web Pages               |Serge Sharoff    |Politecnico di Bari  |
|On the Design of Local Network Interfaces.                    |David J. Farber  |Politecnico di Milano|
+--------------------------------------------------------------+-----------------+---------------------+



DataFrame[affiliation: string, bio: string, email: string, authorId: bigint, name: string]

Return the number of publications that have at least one fos whose name contains "Computer"

In [90]:
# WHERE, IN Nested Query
computerScience_fos = fos_df.filter(col("name").contains("Computer")).select(col("id")).collect()
computerScience_fos = [csf[0] for csf in computerScience_fos]
count_cs_publications = rel_df.filter(col("fos_id").isin(computerScience_fos)).select(col("pub_id")).distinct().count()
print("Percentage of publications that have fos with Computer: " + str(count_cs_publications/2500*100) + '%')

Percentage of publications that have fos with Computer: 80.4%


The query counts for each name of the venue (that has different ID since it can have different editions or volumes) the number of papers that were presented there

In [91]:
# GROUP BY, 1 JOIN, AS
venue_df.join(publication_df, venue_df.id == publication_df.venue, "inner")\
    .groupby("raw")\
        .count()\
            .select(venue_df.raw, col("count").alias("Number of papers for every raw")).show()

+--------------------+------------------------------+
|                 raw|Number of papers for every raw|
+--------------------+------------------------------+
|The Journal of Lo...|                             1|
|International Con...|                            16|
|            Contexts|                             1|
|            CARTOCON|                             1|
|                ITSL|                             1|
|International Con...|                             6|
|International Wor...|                             1|
|5th Workshop on C...|                             1|
|ACM Sigmod Digita...|                             1|
|Logic programming...|                             1|
|               IOLTW|                             1|
|Computer Supporte...|                             4|
|IS&N '95 Proceedi...|                             1|
|International Con...|                             3|
|     CAiSE Workshops|                             1|
|Semantics and Dig...|      

It filters the papers that have at least 3 authors, then it shows for every publisher the max number of pages between the papers he published, shown in descending order

In [92]:
# WHERE, GROUP BY
publication_df.filter(size('authors') >= 3)\
        .groupBy('publisher').max('pages')\
            .select(publication_df.publisher, col("max(pages)").alias("Maxpages"))\
                .orderBy(col("Maxpages").desc()).show(50)

+--------------------+--------+
|           publisher|Maxpages|
+--------------------+--------+
|   PoliPrint, Milano|      20|
|                  GI|      20|
|  USENIX Association|      20|
|American Medical ...|      20|
| Springer, Singapore|      20|
|Springer, Berlin,...|      20|
|           IOS Press|      20|
|Morgan Kaufmann P...|      20|
|Society for Compu...|      20|
|Australian Comput...|      20|
|    Springer, London|      20|
|      Springer, Cham|      20|
|                 USA|      20|
|Springer Berlin H...|      19|
|Springer Netherlands|      19|
|Texas A&M University|      19|
|Eurographics Asso...|      19|
|        EDP-Sciences|      19|
|Michigan Publishi...|      19|
|Kluwer Academic P...|      19|
|                IIIS|      19|
|            Springer|      19|
|Elsevier Science ...|      19|
|Springer, Boston, MA|      19|
|            Aka GmbH|      18|
|          SCS Europe|      18|
|Association for C...|      18|
|     RMIT University|      18|
|Departm

The query groups the venues by affiliation, for each of them it counts the number of authors by id and collects the names of the authors in a list, then it filters (HAVING) the affiliations by the number of authors in the list that should be between 5 and 15. We order the table by descending order for the number of authors, and in case of tie they show the affiliation name in alphabetic order

In [93]:
# GROUP BY, HAVING, AS
author_df.groupBy("affiliation")\
        .agg(
            countDistinct("id").alias("Number of Authors"),
            collect_list(author_df.name).alias("Authors list")
            )\
                .filter((col("Number of Authors") < 15) & (col("Number of Authors") > 5))\
                    .orderBy(col("Number of Authors").desc(), col("affiliation").asc())\
                        .show(truncate=True)

+--------------------+-----------------+--------------------+
|         affiliation|Number of Authors|        Authors list|
+--------------------+-----------------+--------------------+
|CHINESE ACADEMY O...|               12|[Dengguo Feng, Ch...|
|Stanford, University|               12|[Edward H. Shortl...|
|Harbin Institute ...|               11|[Wangmeng Zuo, Ho...|
|RWTH Aachen Unive...|               11|[Nicolas R. Gauge...|
|Microsoft Researc...|                9|[John R. Douceur,...|
|Northeastern, Uni...|                8|[Alireza Khalafi,...|
|Regenstrief Insti...|                8|[Gunther Schadow,...|
|Faculty of System...|                7|[Hirokazu Taki, F...|
|Humboldt-Universi...|                7|[Mathias Nitzsche...|
|Nanyang Technolog...|                7|[Yin-Leng Theng, ...|
|National Universi...|                7|[Qaiser Mehmood, ...|
|The University Of...|                7|[Michele Turitto,...|
|UNIVERSITY OF AVEIRO|                7|[Iouliia Skliarov...|
|Univers

DESCRIZIONE QUERY

In [94]:
# WHERE, GROUP BY, HAVING, AS


Count the publications grouping by the type of the venue they are taken from ('C' for Conference, 'J' for Journal). The publications counted are only the ones with at least one author from the 'Politecnico di Bari'.

In [95]:
# WHERE, Nested Query(i.e., 2-step Queries), GROUP BY
authors_from_bari = author_df.filter(col('affiliation') == 'Politecnico di Bari').select('id').collect()
authors_from_bari = [barese[0] for barese in authors_from_bari]

exploded_pub = publication_df.select(publication_df.id, publication_df.year, publication_df.venue, explode(publication_df.authors))
exploded_pub = exploded_pub.withColumnRenamed("col", "author")
exploded_pub = exploded_pub.withColumnRenamed("id", "pub_id")

exploded_pub.filter(col('author').isin(authors_from_bari))\
    .join(venue_df, exploded_pub.venue == venue_df.id, "inner")\
    .groupBy('type').agg(count('pub_id').alias('n_doc')).show()

AttributeError: 'DataFrame' object has no attribute 'year'

From all the publications wrote before the 2000s, compute the average weight of the weights of its Field of Studies and display the top five of them, having the average above 0.5. 

In [ ]:
# WHERE, GROUP BY, HAVING, 1 JOIN
publication_df.filter(col('year')<2000)\
    .join(rel_df, publication_df.id == rel_df.pub_id, "inner")\
    .groupBy('id').agg(avg('weight').alias('avg_weight')).filter(col('avg_weight')>0.5)\
    .orderBy(col('avg_weight').desc())\
    .limit(5).show()

+-------+------------------+
|     id|        avg_weight|
+-------+------------------+
|1840116|            0.7599|
| 683650|          0.744615|
|1787282|          0.735108|
|1645932|         0.7261075|
|1546711|0.6944836363636363|
+-------+------------------+



Count the number of publications, grouping them by publisher, having the Field of Study dealing with Computers (its name must contain the word 'Computer')

In [ ]:
# WHERE, GROUP BY, HAVING, 2 JOINs
fos_df.filter(col('name').contains('Computer'))\
    .join(rel_df, fos_df.id == rel_df.fos_id, "inner")\
    .join(publication_df, rel_df.pub_id == publication_df.id, "inner")\
    .groupBy('publisher').agg(countDistinct('pub_id').alias('n_pub')).filter(col('n_pub')>25)\
    .orderBy(col('n_pub').desc())\
    .limit(5).show()

+--------------------+-----+
|           publisher|n_pub|
+--------------------+-----+
|   PoliPrint, Milano| 1046|
|Springer, Berlin,...|  459|
|      Springer, Cham|   95|
|Morgan Kaufmann P...|   32|
+--------------------+-----+

